In [11]:
import platform
import os

import torch

In [14]:
os.environ["COMPUTERNAME"], platform.node()

('WSLDFCB344', 'WSLDFCB344')

In [15]:
platform.system(), platform.processor()

('Windows', 'Intel64 Family 6 Model 170 Stepping 4, GenuineIntel')

In [16]:
torch.cuda.is_available()

False

In [22]:
try:
    print(os.environ["CUDA_PATH"])
except KeyError:
    print(f"CUDA is not available on {os.environ["COMPUTERNAME"]}!")

CUDA is not available on WSLDFCB344!


In [26]:
os.environ.get("PATH").split(';')

['C:\\Program Files\\PowerShell\\7',
 'C:\\Program Files\\Python313\\Scripts\\',
 'C:\\Program Files\\Python313\\',
 'C:\\WINDOWS\\system32',
 'C:\\WINDOWS',
 'C:\\WINDOWS\\System32\\Wbem',
 'C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\',
 'C:\\WINDOWS\\System32\\OpenSSH\\',
 'C:\\Program Files\\Microsoft VS Code\\bin',
 'C:\\Program Files\\R\\R-4.4.3\\bin',
 'C:\\Program Files\\LLVM\\bin',
 'C:\\Program Files\\PowerShell\\7\\',
 'C:\\Users\\90963425\\AppData\\Local\\Microsoft\\WindowsApps']